In [9]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

In [10]:
articles = pd.read_csv("articles_cleaned.csv")
customers = pd.read_csv('customers_cleaned.csv')
transactions = pd.read_csv('transactions_cleaned.csv')

In [11]:

# Step 1: Sample transactions
sampled_users = transactions['customer_id'].sample(n=500, random_state=42).unique()
sampled_transactions = transactions[transactions['customer_id'].isin(sampled_users)]

# Step 2: Train-Test Split
def train_test_split(transactions, test_size=0.2):
    test = transactions.sample(frac=test_size, random_state=42)
    train = transactions.drop(test.index)
    return train, test

# Perform train-test split on the sampled transactions
train_transactions, test_transactions = train_test_split(sampled_transactions)

# Step 3: Create User-Item Matrices for Train and Test
train_matrix = train_transactions.pivot_table(
    index='customer_id',
    columns='article_id',
    values='price',
    aggfunc='sum'
).fillna(0)

test_matrix = test_transactions.pivot_table(
    index='customer_id',
    columns='article_id',
    values='price',
    aggfunc='sum'
).fillna(0)

# Align test matrix with train matrix
test_matrix = test_matrix.reindex(index=train_matrix.index, columns=train_matrix.columns, fill_value=0)

# Convert matrices to sparse format for efficiency
train_matrix_sparse = csr_matrix(train_matrix.values)
test_matrix_sparse = csr_matrix(test_matrix.values)

# Step 4: Compute User-User Similarity on Train Matrix
train_similarity = cosine_similarity(train_matrix_sparse)
train_similarity_df = pd.DataFrame(train_similarity, index=train_matrix.index, columns=train_matrix.index)

# Step 5: Predict Ratings Based on Train Matrix
def predict_ratings(train_matrix_df, similarity_matrix):
    # Weighted sum of ratings
    weighted_sum = similarity_matrix.dot(train_matrix_df.values)
    # Sum of absolute similarities
    sum_of_similarities = np.abs(similarity_matrix).sum(axis=1).reshape(-1, 1)
    # Predicted ratings
    predicted_matrix = weighted_sum / sum_of_similarities
    return pd.DataFrame(
        predicted_matrix,
        index=train_matrix_df.index,
        columns=train_matrix_df.columns
    )

# Call the function with train_matrix (DataFrame)
predicted_ratings = predict_ratings(train_matrix, train_similarity)



# Step 6: Flatten Matrices for Evaluation
def flatten_for_evaluation(predicted_matrix, actual_matrix):
    mask = actual_matrix > 0  # Consider only actual interactions
    predicted_flat = predicted_matrix[mask]
    actual_flat = actual_matrix[mask]
    return predicted_flat, actual_flat

predicted_flat, actual_flat = flatten_for_evaluation(
    pd.DataFrame(predicted_ratings, index=train_matrix.index, columns=train_matrix.columns).values,
    test_matrix.values
)

# Step 7: Calculate RMSE and MAE
rmse = np.sqrt(mean_squared_error(actual_flat, predicted_flat))
mae = mean_absolute_error(actual_flat, predicted_flat)

# Step 8: Output the Results
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")


RMSE: 0.031171138718658366
MAE: 0.024377739499250706
